In [79]:
from investpy.stocks import get_stock_historical_data as gshd
from datetime import datetime, timedelta
import pandas as pd
import os

In [80]:
df = pd.read_csv('.\\articles_us.csv', encoding='utf-8', sep=',')

In [81]:
companies = list(df.target.drop_duplicates())
dates = sorted(list(date.split('-') for date in df.date.drop_duplicates()))
start, end = '/'.join(dates[0][::-1]), '/'.join(dates[-1][::-1])

In [82]:
codes = [
    'ASML', 'ADBE', 'GOOG', 'AMZN', 'AMGN', 'AAPL', 'CSCO', 'CMCSA', 'COST', 'FB', 'INTC', 'MSFT', 'NVDA', 'NFLX', 'PYPL', 'PEP'
]

In [83]:
for i in range(len(codes)):
    price = gshd(codes[i], 'united states', start, end)
    price.to_csv(f'.\\nasdaq\\{companies[i]}-{codes[i]}.csv', encoding='utf-8', sep=',')

In [84]:
nasdaq_route = '.\\nasdaq\\'
datas = os.listdir(nasdaq_route)

In [85]:
dfs, names = {}, []
for data in datas:
    csv_route = nasdaq_route + data
    csv_name = data.split('-')
    if len(csv_name) > 2:
        csv_name = '-'.join(csv_name[:2])
    else:
        csv_name = csv_name[0]
    
    df = pd.read_csv(csv_route, encoding='utf-8', sep=',')
    dfs[csv_name] = df
    names.append(csv_name)

In [86]:
articles = pd.read_csv('.\\articles_us.csv', encoding='utf-8', sep=',')
co_articles = []
for name in names:
    sep_articles = articles[articles['target'] == name]
    co_articles.append(sep_articles)

In [87]:
updown = {}
for co in co_articles:
    co_dates = list(co['date'])
    updown[list(co['target'].drop_duplicates())[0]] = co_dates

In [96]:
data = [[], []]
columns = ['date', 'updown']
for i in range(len(dfs)):
    nasdaq, target = sorted(dfs.keys())[i], sorted(updown.keys())[i]
    money, dates = dfs[nasdaq], updown[target]
    nasdaq_dates = list(money.Date.drop_duplicates())
    for date in dates:
        if date in nasdaq_dates:
            idx = nasdaq_dates.index(date)
            if idx != len(nasdaq_dates) - 1:
                diff = money.iloc[idx, 3] - money.iloc[idx, 1]
                if diff >= 0:
                    data[1].append(1)
                else:
                    data[1].append(0)
                data[0].append(date)
            else:
                data[1].append(1)
                data[0].append(date)
        else:
            for compare in nasdaq_dates:
                real_compare = datetime.strptime(compare, '%Y-%m-%d')
                real_date = datetime.strptime(date, '%Y-%m-%d')
                if abs(real_compare - real_date) <= timedelta(days=1):
                    idx = nasdaq_dates.index(compare)
                    break
            if idx != 0:
                diff = money.iloc[idx, 3] - money.iloc[idx - 1, 1]
                if diff >= 0:
                    data[1].append(1)
                else:
                    data[1].append(0)
                data[0].append(date)
            else:
                data[1].append(1)
                data[0].append(date)

In [97]:
new_df = pd.DataFrame(data).T
new_df.columns = columns

In [98]:
new_df.set_index('date', inplace=True)
new_df.to_csv('.\\label_us.csv', encoding='utf-8', sep=',')